## 字符串输出解析器：StrOutputParser

In [2]:
# 1. 获取大模型
import os
import dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 2. 调用大模型
response = chat_model.invoke("什么是币安币？简要概述")
print(type(response))

# 3. 获取字符串的输出结果
# print(response.content)

parser = StrOutputParser()
output_str_response = parser.invoke(response)
print(output_str_response)

<class 'langchain_core.messages.ai.AIMessage'>
币安币（Binance Coin，简称BNB）是由全球最大的加密货币交易所之一币安（Binance）发行的数字资产。最初，BNB是在以太坊区块链上发行的ERC-20标准代币，随后在2019年币安推出了自己的区块链——币安智能链（Binance Smart Chain），BNB也迁移到了该区块链。

BNB的主要用途包括：

1. **交易手续费折扣**：用户在币安交易所使用BNB支付交易手续费时，可以享受折扣。
2. **参与IEO**：BNB可以用于参与币安平台的首次交易所发行（IEO），为新项目的筹资提供便利。
3. **DeFi应用**：在币安智能链上，BNB可用于参与去中心化金融（DeFi）应用，提供流动性等。
4. **其他生态系统应用**：BNB还可以用于币安生态系统中的各种服务和应用，如购物、支付等。

随着币安平台和生态系统的发展，BNB的应用场景不断扩大，使其成为一种重要的加密货币。


## JsonOutputParser JSON解析器


In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个十分靠谱的{role}专家"),
    ("human", "{question}")
])

prompt = chat_prompt_template.invoke(input={"role": "数学家", "question": "解释一下什么是傅里叶变换？返回结果使用JSON格式输出"})

response = chat_model.invoke(prompt)

# 获取一个JsonOutputParser
parser = JsonOutputParser()

# 如果不加上以JSON格式输出的Prompt提示，会报错，原因是输出的response.content是一个字符串
json_result = parser.invoke(response)
print(json_result)

{'傅里叶变换': {'定义': '傅里叶变换是一种数学变换，用于分析和表示信号在频率域中的特性。它将时间域的信号转换为频率域，帮助我们理解信号的频谱成分。', '用途': ['信号处理', '图像处理', '通讯系统', '音频分析', '振动分析', '量子物理'], '数学表达': {'连续傅里叶变换': {'公式': 'F(ω) = ∫ f(t) e^{-jωt} dt', '说明': 'f(t) 是时间域信号，ω 是频率变量，j 是虚数单位。'}, '离散傅里叶变换': {'公式': 'X[k] = Σ x[n] e^{-j(2π/N)kn}', '说明': 'x[n] 是离散信号，N 是信号长度，k 是频率索引。'}}, '特点': ['线性：输入信号的线性组合对应于输出信号的线性组合。', '时变性：时间域信号的时间平移会导致频域信号的相位变化。', '卷积定理：时域信号的卷积对应于频域信号的乘法。']}}


In [6]:
json_parser = JsonOutputParser()
print(json_parser.get_format_instructions())

Return a JSON object.


In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")

joke_query = "请讲一个笑话"

# 定义解析器
parser = JsonOutputParser()

# PromptTemplate为示例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为：{format_instructions}\n 问题是：{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

prompt = prompt_template.invoke(input={"question": joke_query})
response = chat_model.invoke(prompt)
print(response.content)
print("\n")
json_response = parser.invoke(response)
print(json_response)

```json
{
  "joke": "为什么数学书总是感到忧伤？因为它有太多的问题！"
}
```


{'joke': '为什么数学书总是感到忧伤？因为它有太多的问题！'}


### 管道符号 |

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")

joke_query = "请讲一个笑话"

# 定义解析器
parser = JsonOutputParser()

# PromptTemplate为示例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为：{format_instructions}\n 问题是：{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# prompt = prompt_template.invoke(input={"question": joke_query})
# response = chat_model.invoke(prompt)
chain = prompt_template | chat_model | parser
output = chain.invoke({"question": joke_query})
print(output)

{'joke': '为什么程序员总是分不清万圣节和圣诞节？因为Oct 31 = Dec 25！'}


## XMLOutputParser

In [10]:
# 使用get_format_instructions()方法
from langchain_core.output_parsers import XMLOutputParser

output_parser = XMLOutputParser()

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```
